## 1. Importing libraries and Dataset Configuration

In [1]:
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import re
from datetime import datetime, timedelta
from collections import defaultdict, deque
import pickle
import psycopg2

/home/haidarhanif18/.conda/envs/haidarenv/lib/python3.9/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print('gpu ', gpu)
    tf.config.experimental.set_memory_growth(gpu, True)

gpu  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


## 2. Database Connection and Retrieve

In [3]:
# Define the connection parameters
db_params = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'AZX)$aPNH4zpDrj}',
    'host': '34.173.222.240',
    'port': '5432'
}

# Connect to PostgreSQL database
try:
    conn = psycopg2.connect(**db_params)
    print("Connected to the database")
except Exception as e:
    print(f"Unable to connect to the database: {e}")

Connected to the database


In [4]:
query = "SELECT sentences, label_task FROM tbl1;"
df = pd.read_sql_query(query, conn)
df

/tmp/ipykernel_686863/2806474399.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,sentences,label_task
0,buat web front end,Frontend Development\n
1,bikin frontend,Frontend Development
2,buat front end website,Frontend Development
3,bikin front end bagus,Frontend Development
4,front end,Frontend Development
5,website front end,Frontend Development
6,front website,Frontend Development


In [5]:
conn.close()
print("Connection closed")

Connection closed


In [8]:
# Filter labels with at least 45 sentences
label_counts = df['label_task'].value_counts()
valid_labels = label_counts[label_counts >= 40].index

filtered_df = df[df['label_task'].isin(valid_labels)]
num_classes = filtered_df['label_task'].nunique()

In [9]:
label_counts = filtered_df['label_task'].value_counts()
print(label_counts)

Series([], Name: count, dtype: int64)


In [7]:
import tensorflow as tf

# Load the saved model
model = tf.keras.models.load_model('text_classify.h5')

# Load the saved tokenizer and label encoder
with open('tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

with open('label_encoder.pkl', 'rb') as handle:
    label_encoder = pickle.load(handle)

In [16]:
# Preprocess and tokenize the new data
new_sentences = filtered_df['sentences'].tolist()
new_labels = filtered_df['label_task'].tolist()

new_sequences = tokenizer.texts_to_sequences(new_sentences)
new_padded = pad_sequences(new_sequences, maxlen=100, padding='post', truncating='post')

new_labels_encoded = label_encoder.transform(new_labels)
new_labels_one_hot = tf.keras.utils.to_categorical(new_labels_encoded, num_classes=len(label_encoder.classes_))

In [17]:
# Fine-tune the loaded model with the new data
history = model.fit(new_padded, 
                    new_labels_one_hot, 
                    epochs=5,  # A few epochs for fine-tuning
                    validation_split=0.2)

Epoch 1/5


2024-06-14 07:16:45.911942: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400
2024-06-14 07:16:49.247077: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-06-14 07:16:49.251794: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-06-14 07:16:49.251880: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-06-14 07:16:49.253654: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-06-14 07:16:49.253733: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-06-14 07:16:49.419514: I tensorflow/stream_executor/cuda/c

1/1 [==============================] - 11s 11s/step - loss: 0.1694 - accuracy: 1.0000 - val_loss: 0.4143 - val_accuracy: 1.0000
Epoch 2/5
1/1 [==============================] - 0s 128ms/step - loss: 0.6915 - accuracy: 0.7500 - val_loss: 0.3769 - val_accuracy: 1.0000
Epoch 3/5
1/1 [==============================] - 0s 115ms/step - loss: 0.1685 - accuracy: 1.0000 - val_loss: 0.3438 - val_accuracy: 1.0000
Epoch 4/5
1/1 [==============================] - 0s 118ms/step - loss: 0.2527 - accuracy: 1.0000 - val_loss: 0.3112 - val_accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 0s 282ms/step - loss: 0.5511 - accuracy: 0.7500 - val_loss: 0.2801 - val_accuracy: 1.0000


In [ ]:
# Save the updated model
model.save('text_classify_feedbacked.h5')